In [1]:
import csv
import pymysql
import requests
import bs4

conn = pymysql.connect(host = 'localhost', user ='root', password = '111111',
                      db = 'shoppingdb', charset = 'utf8')

cur = conn.cursor()

f = open('koreaStock.csv','r')

csvReader = list(csv.reader(f))

In [2]:
sql_code = '''SELECT code FROM company_info;'''
cur.execute(sql_code)
result = cur.fetchall()

In [3]:
""" url = 'https://finance.naver.com/item/sise_day.nhn?code='f'{result[12][0]}' 
res = requests.get(url, headers = {'User-agent': 'Mozilla/5.0'})

bs_obj = bs4.BeautifulSoup(res.content, "html.parser")

stock_data = bs_obj.find_all("td", {"class" : "num"})
stock_data_list = [bs_obj.find_all("span")[n].string for n in range(0, len(stock_data)+11)] """

In [4]:
import requests
import bs4

def importdata(start, end, page):
    for i in range(start, end): # len(result)
        try:
            url = 'https://finance.naver.com/item/sise_day.nhn?code='f'{result[i][0]}&page={page}'
            res = requests.get(url, headers = {'User-agent': 'Mozilla/5.0'})

            bs_obj = bs4.BeautifulSoup(res.content, "html.parser")

            stock_data = bs_obj.find_all("td", {"class" : "num"})
            stock_data_list = [bs_obj.find_all("span")[n].string for n in range(0, len(stock_data) + 11)]
        except:
            pass
        
        k = 1
        try:
            for j in range(0,10):
            
                code = result[i][0]
                date = stock_data_list[k]
                close = int(stock_data_list[k+1].replace(',',''))
                diff = int(stock_data_list[k+2].replace(',',''))
                opens = int(stock_data_list[k+3].replace(',',''))
                high = int(stock_data_list[k+4].replace(',',''))
                low = int(stock_data_list[k+5].replace(',',''))
                vol = int(stock_data_list[k+6].replace(',',''))
    
                sql = '''insert into daily_price (code, date, open, high, low, close, diff, volume) values (%s, %s, %s, %s, %s, %s, %s, %s)'''
                cur.execute(sql, (code, date, opens , high, low, close, diff, vol))
                k += 7
        except:
            return None

    conn.commit()

In [5]:
div_code_100 = [i for i in range(0, len(result), 100)]
div_code_100.append(len(result))

In [ ]:
for i in range(len(div_code_100)-1):
    importdata(div_code_100[i], div_code_100[i+1])
    print('%d개 종목 데이터를 정상적으로 DB에 집어넣었습니다.'% (div_code_100[i+1]))